In [8]:
from mtcnn import MTCNN
import cv2
import torch
from torchvision import transforms
from facenet_pytorch import MTCNN, InceptionResnetV1
import os
import numpy as np


In [9]:
from mtcnn.mtcnn import MTCNN
import cv2
import os

# Initialize the MTCNN detector
detector = MTCNN()

# Path to your image folder
image_folder = 'Database_of_Images'

# List to store detected faces
faces = []

# Loop through images and detect faces
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder, filename))
    
    # Detect faces using the MTCNN detector
    result = detector.detect_faces(img)
    
    if result:
        for face_info in result:
            x, y, width, height = face_info['box']
            face = img[y:y+height, x:x+width]
            faces.append(face)


1/1 [==============================] - 0s 28ms/step


In [39]:

# Initialize the pre-trained InceptionResnetV1 model
resnet = InceptionResnetV1(pretrained='vggface2').eval()

# Define the transformation to apply to your face images
transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize((160, 160)),transforms.ToTensor(), transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),])

encodings = []
for image in faces:
    image = transform(image)
    image = image.unsqueeze(0)
    image = np.around(image, decimals=12)
    encoding = resnet(image)
    encoding /= torch.norm(encoding, p=2)
    encodings.append(encoding)

In [40]:
new_img = cv2.imread('Test_database/waqar_test.jpg')
new_result = detector.detect_faces(new_img)


# Check if a face was detected
if new_result:
    # Get the coordinates of the first detected face
    x, y, width, height = new_result[0]['box']

    # Extract the face region from the image
    detected_face = new_img[y:y+height, x:x+width]

    # Now, 'detected_face' contains the pixel values of the detected face
else:
    print("No face detected in the image.")


1/1 [==============================] - 0s 26ms/step


In [41]:
# Display the detected face using OpenCV
cv2.imshow('Detected Face', detected_face)
cv2.waitKey(0)  # Wait until a key is pressed
cv2.destroyAllWindows()  # Close the OpenCV window

In [42]:

image_new = transform(detected_face)
image_new = image_new.unsqueeze(0)
image_new = np.around(image_new, decimals=12)
new_encoding = resnet(image_new)
new_encoding /= torch.norm(new_encoding, p=2)

In [43]:
threshold = 0.999999

# Initialize variables for minimum distance
min_dist = float('inf')

# Initialize match as False
match = False

# Calculate the distance between the target encoding and each reference encoding
distances = []
for ref_encoding in encodings:
    dist = torch.norm(new_encoding - ref_encoding).detach().numpy()
    distances.append(dist)

# Check if any of the distances are less than the threshold
for dist in distances:
    print(dist)
    if dist <= threshold:
        match = True  # Match found

if match:
    print("Match found.")
else:
    print("No match found.")


1.4719193
1.4120965
1.3655161
1.4200542
1.304007
1.5101974
1.333215
1.3843735
1.3987107
1.0863696
1.4502021
0.8886956
0.28992715
Match found.
